In [1]:
import gc
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm
import optuna

In [2]:
%load_ext autoreload
%autoreload 2

# drGAT パッケージのパスを追加
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(parent_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from drGAT import drGAT
from drGAT.load_data import load_data
from drGAT.metrics import compute_metrics_stats
from drGAT.sampler import NewSampler
from drGAT.utility import filter_target

/data/inouey2/conda/envs/genex/lib/python3.10/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.29' not found (required by /gpfs/gsfs12/users/inouey2/conda/envs/genex/lib/python3.10/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/data/inouey2/conda/envs/genex/lib/python3.10/site-packages/torch_geometric/typing.py:72: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /gpfs/gsfs12/users/inouey2/conda/envs/genex/lib/python3.10/site-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZN2at23SavedTensorDefaultHooks11set_tracingEb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/data/inouey2/conda/envs/genex/lib/python3.10/site-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. S

In [3]:
data = "nci"
method = 'GAT'
target_dim = 0  # Cell
# target_dim = 1  # Drug
(
    res,
    null_mask,
    S_d,
    S_c,
    S_g,
    drug_feature,
    gene_norm_gene,
    gene_norm_cell,
    A_cg,
    A_dg,
) = load_data(data)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

load nci
Done!


In [4]:
def drGAT_new(
    res_mat,
    null_mask,
    target_dim,
    target_index,
    S_d,
    S_c,
    S_g,
    A_cg,
    A_dg,
    params,
    device,
):
    sampler = NewSampler(
        res,
        null_mask,
        target_dim,
        target_index,
        S_d,
        S_c,
        S_g,
        A_cg,
        A_dg,
    )

    (_, _, _, best_val_labels, best_val_prob, best_metrics, _, _, _) = drGAT.train(
        sampler, params=params, device=device, verbose=False
    )

    return best_val_labels, best_val_prob

In [5]:
def suggest_hyperparams(trial, S_d, S_c, S_g):
    hidden1 = trial.suggest_int("hidden1", 256, 512)
    hidden2 = trial.suggest_int("hidden2", 64, min(256, hidden1))
    hidden3 = trial.suggest_int("hidden3", 32, min(128, hidden2))

    final_mlp_layers = trial.suggest_int("final_mlp_layers", 1, 3)

    params = {
        "n_drug": S_d.shape[0],
        "n_cell": S_c.shape[0],
        "n_gene": S_g.shape[0],
        "dropout1": trial.suggest_float("dropout1", 0.1, 0.5, step=0.1),
        "dropout2": trial.suggest_float("dropout2", 0.1, 0.5, step=0.1),
        "dropout3": (
            trial.suggest_float("dropout3", 0.1, 0.5, step=0.1)
            if final_mlp_layers >= 2
            else 0.0
        ),
        "hidden1": hidden1,
        "hidden2": hidden2,
        "hidden3": hidden3,
        "epochs": 1,
        "heads": 1,
        # "epochs": trial.suggest_int("epochs", 300, 1000, step=100),
        # "heads": trial.suggest_int("heads", 2, 8),
        "activation": trial.suggest_categorical("activation", ["relu", "gelu"]),
        "optimizer": trial.suggest_categorical("optimizer", ["Adam", "AdamW"]),
        "lr": trial.suggest_float("lr", 1e-5, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True),
        "scheduler": trial.suggest_categorical("scheduler", [None, "Cosine"]),
        "norm_type": trial.suggest_categorical(
            "norm_type", ["GraphNorm", "BatchNorm", "LayerNorm"]
        ),
        "gnn_layer": method,
        "final_mlp_layers": final_mlp_layers,
        # "residual": trial.suggest_categorical("residual", [True, False]),
        "attention_dropout": trial.suggest_float(
            "attention_dropout", 0.0, 0.4, step=0.1
        ),
    }

    if params["scheduler"] == "Cosine":
        min_epoch_div = max(1, params["epochs"] // 5)
        max_epoch_div = max(min_epoch_div + 1, params["epochs"] // 2)
        params["T_max"] = trial.suggest_int(
            "T_max", low=min_epoch_div, high=max_epoch_div
        )

        if params["T_max"] <= 0:
            raise optuna.TrialPruned(f"Invalid T_max: {params['T_max']}")

    return params

In [6]:
def handle_optuna_errors(e, trial):
    msg = str(e)
    if "CUDA out of memory" in msg:
        print(f"Pruned trial {trial.number}: CUDA OOM")
        with torch.cuda.device("cuda"):
            torch.cuda.empty_cache()
        gc.collect()
        raise optuna.TrialPruned(f"OOM at trial {trial.number}")
    elif "Input contains NaN" in msg:
        print(f"Pruned trial {trial.number}: Input contains NaN")
        raise optuna.TrialPruned(f"NaN input at trial {trial.number}")
    elif isinstance(e, ZeroDivisionError):
        print(f"Pruned trial {trial.number}: ZeroDivisionError in CosineAnnealingLR")
        raise optuna.TrialPruned("ZeroDivisionError in CosineAnnealingLR")
    else:
        print(f"Unexpected error in trial {trial.number}: {msg}")
        raise e

In [7]:
def objective(trial):
    try:
        # === Data Load ===
        is_zero_pad = trial.suggest_categorical("is_zero_pad", [True, False])
        (
            drugAct,
            null_mask,
            S_d,
            S_c,
            S_g,
            drug_feature,
            gene_norm_gene,
            gene_norm_cell,
            A_cg,
            A_dg,
        ) = load_data(data, is_zero_pad=is_zero_pad)

        # === Suggest Hyperparameters ===
        params = suggest_hyperparams(trial, S_d, S_c, S_g)

        true_datas = pd.DataFrame()
        predict_datas = pd.DataFrame()
        
        samples = res.shape[target_dim]
        
        passed_targets = []
        skipped_targets = []
        
        for target_index in range(samples):
            label_vec = res.iloc[target_index] if target_dim == 0 else res.iloc[:, target_index]
            passed, reason, pos, neg, total = filter_target(label_vec)
        
            if passed:
                passed_targets.append(target_index)
            else:
                skipped_targets.append((target_index, reason, pos, neg, total))
        
        # --- passしなかったやつを表示 ---
        print(f"\n🚫 Skipped Targets: {len(skipped_targets)}")
        for idx, reason, pos, neg, total in skipped_targets:
            print(f"Target {idx}: skipped because {reason} (total={total}, pos={pos}, neg={neg})")
        
        # --- ここから学習 ---
        true_datas = pd.DataFrame()
        predict_datas = pd.DataFrame()
        
        for target_index in tqdm(passed_targets):
            true_data, predict_data = drGAT_new(
                res_mat=res,
                null_mask=null_mask.values,
                target_dim=target_dim,
                target_index=target_index,
                S_d=S_d,
                S_c=S_c,
                S_g=S_g,
                A_cg=A_cg,
                A_dg=A_dg,
                params=params,
                device=device,
            )
        
            true_datas = pd.concat([true_datas, pd.DataFrame(true_data).T], ignore_index=True)
            predict_datas = pd.concat([predict_datas, pd.DataFrame(predict_data).T], ignore_index=True)

        # === Compute Final Metrics ===
        metrics_result = compute_metrics_stats(
            trial=trial,
            true=true_datas,
            pred=predict_datas,
            target_metrics=["AUROC", "AUPR", "F1", "ACC"],
        )
        return tuple(metrics_result["target_values"])

    except Exception as e:
        handle_optuna_errors(e, trial)

In [8]:
study = optuna.create_study(
    directions=["maximize"] * 4,
    sampler=optuna.samplers.NSGAIISampler(),
    pruner=optuna.pruners.HyperbandPruner(),
    storage=f"sqlite:///{method}_{data}_{'cell' if target_dim == 0 else 'drug'}.sqlite3",
    study_name=method,
    load_if_exists=True,
)
study.optimize(objective, n_trials=1000)

[I 2025-05-12 16:23:01,795] A new study created in RDB with name: GAT


load nci
Done!

🚫 Skipped Targets: 2
Target 15: skipped because low_positive_ratio (total=488, pos=7, neg=481)
Target 33: skipped because low_negative_ratio (total=427, pos=419, neg=8)


  0%|          | 0/57 [00:00<?, ?it/s]

Using device: cuda


/spin1/home/linux/inouey2/drGAT/drGAT/drGAT.py:308: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if use_autocast else None
  2%|▏         | 1/57 [00:05<05:03,  5.42s/it]

...

/spin1/home/linux/inouey2/drGAT/drGAT/drGAT.py:308: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if use_autocast else None
 51%|█████     | 29/57 [00:16<00:15,  1.78it/s]

Using device: cuda
Best model found at epoch 1


 51%|█████     | 29/57 [00:16<00:16,  1.73it/s]
[W 2025-05-12 16:23:59,663] Trial 1 failed with parameters: {'is_zero_pad': True, 'hidden1': 328, 'hidden2': 104, 'hidden3': 70, 'final_mlp_layers': 1, 'dropout1': 0.5, 'dropout2': 0.4, 'activation': 'gelu', 'optimizer': 'Adam', 'lr': 0.005194664958916714, 'weight_decay': 0.005033508995929664, 'scheduler': 'Cosine', 'norm_type': 'GraphNorm', 'attention_dropout': 0.2, 'T_max': 2} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/data/inouey2/conda/envs/genex/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1790637/2727140603.py", line 48, in objective
    true_data, predict_data = drGAT_new(
  File "/tmp/ipykernel_1790637/2529068810.py", line 26, in drGAT_new
    (_, _, _, best_val_labels, best_val_prob, best_metrics, _, _, _) = drGAT.train(
  File "/spin1/home/linux/inouey2/drGAT/drGAT/drGAT.py", line 325

Using device: cuda



KeyboardInterrupt



In [9]:
study.trials_dataframe()

,number,values_0,values_1,values_2,values_3,datetime_start,datetime_complete,duration,params_T_max,params_activation,...,user_attrs_MCC_mean,user_attrs_MCC_std,user_attrs_Precision_mean,user_attrs_Precision_std,user_attrs_Recall_mean,user_attrs_Recall_std,user_attrs_Specificity_mean,user_attrs_Specificity_std,system_attrs_nsga2:generation,state
0,0,0.498958,0.5125,0.237658,0.420838,2025-05-12 16:23:01.808424,2025-05-12 16:23:41.035921,0 days 00:00:39.227497,NaN,gelu,...,-0.00047,0.039242,0.270527,0.36342,0.457522,0.491733,0.542699,0.496536,0,COMPLETE
1,1,NaN,NaN,NaN,NaN,2025-05-12 16:23:41.067106,2025-05-12 16:23:59.650318,0 days 00:00:18.583212,2.0,gelu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,FAIL
